**Tugas 7 - Keyword Extraction Menggunakan Centrality**

Nama : Isnita Widyur Rahmah
NIM : 220411100048
Kelas : IF 7A

Link Project : https://github.com/taawdyy/ppw

##Install Pustaka Python

In [1]:
!pip install nltk pandas requests beautifulsoup4

#Impor Pustaka

In [2]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Crawling Data Artikel Berita Menggunakan Python
melakukan crawling untuk mengambil judul, isi, tanggal, dan kategori artikel dari URL yang diberikan, dan menyajikan data tersebut dalam bentuk DataFrame untuk kemudahan analisis lebih lanjut.

In [3]:
def crawl_article(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Memastikan permintaan berhasil
        soup = BeautifulSoup(response.content, 'html.parser')

        # Mengambil judul
        title_element = soup.find('h1', class_='text-cnn_black')
        title = title_element.get_text().strip() if title_element else 'Judul tidak ditemukan'

        # Mengambil Isi
        content_div = soup.find('div', class_='detail-text')
        content = "\n".join([p.get_text().strip() for p in content_div.find_all('p')]) if content_div else 'Isi artikel tidak ditemukan'

        # Mengambil tanggal
        date_div = soup.find('div', class_='text-cnn_grey text-sm mb-4')
        date_text = date_div.text.strip() if date_div else 'Tanggal tidak ditemukan'

        # Mengambil kategori
        category_meta = soup.find("meta", attrs={'name': 'dtk:namakanal'})
        category = category_meta['content'].strip() if category_meta and 'content' in category_meta.attrs else 'Kategori tidak ditemukan'

        return {'Judul': title, 'Isi': content, 'Tanggal': date_text, 'Kategori': category}
    except requests.RequestException as e:
        print(f"Error fetching article: {e}")
        return None

article_url = input("Masukkan URL artikel: ")

# Melakukan crawl pada satu artikel
article = crawl_article(article_url)

df = pd.DataFrame([article])  # Membuat DataFrame dari dictionary
df

Masukkan URL artikel: https://www.cnnindonesia.com/olahraga/20241017084015-142-1156275/indonesia-vs-bahrain-momen-krusial-rebutan-tiket-piala-dunia


,Judul,Isi,Tanggal,Kategori
0,"Indonesia vs Bahrain, Momen Krusial Rebutan Ti...",Timnas Indonesia akan menghadapi Bahrain dalam...,"Kamis, 17 Okt 2024 09:32 WIB",olahraga


#Transformasi Teks Menjadi Huruf Kecil
mengubah semua teks dalam kolom Isi menjadi huruf kecil dan menyimpannya dalam kolom baru.

In [4]:
# Fungsi untuk mengubah teks menjadi huruf kecil
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

df['lower case'] = df['Isi'].apply(clean_lower)
df['lower case']

,lower case
0,timnas indonesia akan menghadapi bahrain dalam...


#Pembersihan Teks dari Tanda Baca dan Angka
membersihkan teks dari tanda baca dan angka, serta menghapus karakter yang tidak diinginkan, dan menyimpannya dalam kolom baru.

In [5]:
# Fungsi untuk membersihkan tanda baca dan angka (menghapus koma)
def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z .]+')  # Koma dihilangkan dari pengecualian
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

df['tanda baca'] = df['lower case'].apply(clean_punct)
df['tanda baca']

,tanda baca
0,timnas indonesia akan menghadapi bahrain dalam...


#Normalisasi Spasi dalam Teks
melakukan normalisasi spasi dalam teks dengan mengganti beberapa spasi berturut-turut menjadi satu spasi dan menghapus spasi di awal dan akhir.

In [6]:
# Fungsi untuk normalisasi spasi
def _normalize_whitespace(text):
    if isinstance(text, str):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

df['spasi'] = df['tanda baca'].apply(_normalize_whitespace)
df['spasi']

,spasi
0,timnas indonesia akan menghadapi bahrain dalam...


#Tokenisasi Teks Menjadi Kata
melakukan tokenisasi pada teks yang telah dinormalisasi, memecahnya menjadi daftar kata atau token, dan menyimpannya dalam kolom baru.

In [7]:
# Tokenisasi
def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)
    return text

df['token'] = df['spasi'].apply(tokenize_text)
df['token']

,token
0,"[timnas, indonesia, akan, menghadapi, bahrain,..."


#Pembersihan Stopwords dari Token
membersihkan daftar token dari kata-kata umum yang tidak memberikan makna penting (stopwords), sehingga hanya kata-kata yang relevan yang tersisa untuk analisis lebih lanjut.

In [8]:
# Pembersihan stopwords setelah tokenisasi
def clean_stopwords(tokens):
    if isinstance(tokens, list):
        stopword = set(stopwords.words('indonesian'))
        filtered_tokens = [word for word in tokens if word not in stopword]
        return filtered_tokens
    return tokens

df['stopwords'] = df['token'].apply(clean_stopwords)
df['stopwords']

,stopwords
0,"[timnas, indonesia, menghadapi, bahrain, laga,..."


#Penggabungan Token Menjadi Teks yang Diproses
menggabungkan kembali daftar token yang telah dibersihkan menjadi string utuh, yang kemudian dapat digunakan untuk analisis lebih lanjut.

In [9]:
# Gabungkan kembali token menjadi string
df['processed_text'] = df['stopwords'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else '')

prep_result = df['processed_text'].to_dict()
print(prep_result)

{0: 'timnas indonesia menghadapi bahrain laga kualifikasi piala dunia maret . laga momen krusial perebutan tiket piala dunia . duel timnas indonesia vs bahrain dijadwalkan maret . duel matchday ketujuh alias masuk matchday babak ketiga kualifikasi piala dunia . kondisi indonesia vs bahrain momen krusial tim . tim menjaga peluang lolos piala dunia perebutan terjaga . advertisement scroll to continue with content berkaca kondisi klasemen grup c kualifikasi piala dunia zona asia pertarungan perebutan posisi klasemen ketat . tim tertinggal perebutan posisi peluang finis posisi diyakini terbuka lebar . bahrain perubahan lokasi venue laga timnas indonesia vs bahrain indonesia kerugian skuad garuda . bahrain perubahan lokasi pertandingan alasan keselamatan pemain . mengecam serangan serangan media sosial dilancarkan penggemar timnas indonesia . kekecewaan penggemar timnas indonesia tumpah lantaran insiden kontroversial bahrain vs indonesia . bahrain mencetak gol menit kesembilan injury time i

#Kumpulan Kalimat dan Kata Unik dari Teks Terproses
menyiapkan data untuk analisis lebih lanjut dengan membuat kumpulan kalimat dari teks yang telah diproses dan mengumpulkan kata-kata unik

In [10]:
# Membuat kumpulan kalimat
sentences = []
for p in df['processed_text']:
  sentences.extend(sent_tokenize(p))

# Membuat kumpulan kata unik
vocabulary = set()
for sentence in sentences:
  for word in sentence.split():
    vocabulary.add(word)

print(sentences)
print(vocabulary)

['timnas indonesia menghadapi bahrain laga kualifikasi piala dunia maret .', 'laga momen krusial perebutan tiket piala dunia .', 'duel timnas indonesia vs bahrain dijadwalkan maret .', 'duel matchday ketujuh alias masuk matchday babak ketiga kualifikasi piala dunia .', 'kondisi indonesia vs bahrain momen krusial tim .', 'tim menjaga peluang lolos piala dunia perebutan terjaga .', 'advertisement scroll to continue with content berkaca kondisi klasemen grup c kualifikasi piala dunia zona asia pertarungan perebutan posisi klasemen ketat .', 'tim tertinggal perebutan posisi peluang finis posisi diyakini terbuka lebar .', 'bahrain perubahan lokasi venue laga timnas indonesia vs bahrain indonesia kerugian skuad garuda .', 'bahrain perubahan lokasi pertandingan alasan keselamatan pemain .', 'mengecam serangan serangan media sosial dilancarkan penggemar timnas indonesia .', 'kekecewaan penggemar timnas indonesia tumpah lantaran insiden kontroversial bahrain vs indonesia .', 'bahrain mencetak g

#Membuat DataFrame untuk Frekuensi Kata dalam Kalimat
mengonversi kumpulan kalimat menjadi representasi vektor frekuensi kata dalam bentuk DataFrame.

In [14]:
pd.set_option('future.no_silent_downcasting', True)
df = pd.DataFrame(columns=list(vocabulary), index=sentences)

# Mengisi nilai kosong dengan 0
df = df.fillna(0)  # Fill with 0s

# Menghitung nilai setiap kata pada kalimat
for i, sentence in enumerate(sentences):
  for word in sentence.split():
    df.loc[sentence, word] += 1

# Melihat isi Dataframe
df

,ketat,kualifikasi,pertarungan,kerugian,ketujuh,keselamatan,sepak,pemain,c,gambas,...,to,diterima,matchday,laga,asosiasi,kekecewaan,nasional,maret,mencetak,lebar
timnas indonesia menghadapi bahrain laga kualifikasi piala dunia maret .,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
laga momen krusial perebutan tiket piala dunia .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
duel timnas indonesia vs bahrain dijadwalkan maret .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
duel matchday ketujuh alias masuk matchday babak ketiga kualifikasi piala dunia .,0,1,0,0,1,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
kondisi indonesia vs bahrain momen krusial tim .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tim menjaga peluang lolos piala dunia perebutan terjaga .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
advertisement scroll to continue with content berkaca kondisi klasemen grup c kualifikasi piala dunia zona asia pertarungan perebutan posisi klasemen ketat .,1,1,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
tim tertinggal perebutan posisi peluang finis posisi diyakini terbuka lebar .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
bahrain perubahan lokasi venue laga timnas indonesia vs bahrain indonesia kerugian skuad garuda .,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
bahrain perubahan lokasi pertandingan alasan keselamatan pemain .,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


#Menghitung dan Menampilkan Frekuensi Kata Teratas
menghitung dan menampilkan kata-kata dengan frekuensi tertinggi dari DataFrame, memberikan wawasan tentang kata-kata yang paling umum muncul dalam kumpulan kalimat.

In [15]:
# Menghitung jumlah kemunculan pada tiap kata
word_frequencies = df.sum(axis=0)

# Mengurutkan berdasarkan jumlah kemunculan kata
sorted_word_frequencies = word_frequencies.sort_values(ascending=False)

# Menampilkan kata teratas
keyword = 3

for i, (word, freq) in enumerate(sorted_word_frequencies.items()):
    if i < keyword:
        print(f"Rank {i+1}: {word} (Frequency: {freq})")

Rank 1: . (Frequency: 15)
Rank 2: bahrain (Frequency: 11)
Rank 3: indonesia (Frequency: 9)
